In [13]:
## all imports
import requests
from bs4 import BeautifulSoup
import time
import operator
import socket
#import cPickle
import re # regular expressions

import pandas as pd
from pandas import DataFrame


import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

#This code is to import the libraries we will ptentially be using during the execution of the tasks I have been assigned to
# data analysis and wrangling
import pandas as pd
import pickle
import numpy as np
import random as rnd
from datetime import timedelta
import scipy
from scipy import stats
#Limiting floats output to 3 decimal points
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) 

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# machine learning and metrics
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, make_scorer, mean_squared_error
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, BayesianRidge, ElasticNet, Lasso, SGDRegressor, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.svm import SVC, LinearSVC, SVR, LinearSVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')
import urllib
import urllib.request
import os
from string import ascii_lowercase
import csv

In [14]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [172]:
#"https://www.transfermarkt.co.uk/luis-suarez/leistungsdatendetails/spieler/44352"
ls="https://www.transfermarkt.co.uk/luis-suarez/leistungsdatendetails/spieler/44352"
rs="https://www.transfermarkt.co.uk/roberto-soldado/leistungsdatendetails/spieler/34870"

In [16]:
ls_i="/marcelo-moreno/profil/spieler/54162"
rs_i="/loic-remy/profil/spieler/45121"
playerlist=[]
playerlist.append(ls_i)
playerlist.append(rs_i)

In [47]:
FiveH_href=pd.read_csv('hrefs-players.csv')
playerlist=FiveH_href.iloc[:,0]
playerlist.pop(253)


250    /marcelo-moreno/profil/spieler/54162
251      /luiz-gustavo/profil/spieler/10471
252         /loic-remy/profil/spieler/45121
254      /achille-emana/profil/spieler/7020
255       /albert-riera/profil/spieler/7373
Name: /seydou-doumbia/profil/spieler/75287, dtype: object

In [48]:
def createNewUrl(player):
    player_splitted=player.split("/")
    url="https://www.transfermarkt.co.uk/"+player_splitted[1]+"/leistungsdatendetails/spieler/"+player_splitted[4]
    return(url)

In [49]:
#url_detailed='https://www.transfermarkt.co.uk/luis-suarez/leistungsdatendetails/spieler/44352'

def scrape_url(url_player):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
    response_detailed = requests.get(url, headers=headers)
    time.sleep(10)
    soup = BeautifulSoup(response_detailed.text,"html.parser")
    return(soup)

In [50]:
def wrangling_soup(soup):
    data = []
    table = soup.find('table', attrs={'class':'items'})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return(data)



In [51]:
with open("practice.csv", "w") as f:
    writer = csv.writer(f,delimiter=',')
    for player in playerlist:
        url=createNewUrl(player)
        soup=scrape_url(url)
        data=wrangling_soup(soup)
        
        for i in data:
            i.insert(0,url)
            writer.writerow(i)
            

In [86]:
players_stats=pd.read_csv('practice.csv',  names=["names","season", "league", "apps","PPM","goals","assists", "yellow_1/yellow_2/red", "mins_played"] )

In [87]:
players_stats.head()

,names,season,league,apps,PPM,goals,assists,yellow_1/yellow_2/red,mins_played
0,https://www.transfermarkt.co.uk/etienne-capoue...,18/19,Premier League,19,"1,47",1,2,6 / - / 1,1.710'
1,https://www.transfermarkt.co.uk/etienne-capoue...,18/19,EFL Cup,1,"0,00",1,-,- / - / -,7'
2,https://www.transfermarkt.co.uk/etienne-capoue...,17/18,Premier League,23,"1,13",1,-,3 / - / -,1.541'
3,https://www.transfermarkt.co.uk/etienne-capoue...,17/18,FA Cup,2,"1,50",1,-,- / - / -,135'
4,https://www.transfermarkt.co.uk/etienne-capoue...,17/18,EFL Cup,1,"0,00",1,-,1 / - / -,79'


In [88]:
for i in range(len(players_stats)):
    players_stats['names'][i].split('/')

In [89]:
players_stats['names']=players_stats['names'].str.replace('https://www.transfermarkt.co.uk/','', regex=True)
players_stats['names']=players_stats['names'].str.replace('/leistungsdatendetails/spieler','', regex=True)
players_stats['names']=players_stats['names'].str.replace('-',' ', regex=True)
for state in players_stats['names']:
    if "/" in state:
        row_num = players_stats.loc[players_stats['names']==state].index[0]
        players_stats.at[row_num, 'names'] = state[:state.index("/")]

In [79]:
players_stats['names']

0           etienne-capoue/63494
1           etienne-capoue/63494
2           etienne-capoue/63494
3           etienne-capoue/63494
4           etienne-capoue/63494
5           etienne-capoue/63494
6           etienne-capoue/63494
7           etienne-capoue/63494
8           etienne-capoue/63494
9           etienne-capoue/63494
10          etienne-capoue/63494
11          etienne-capoue/63494
12          etienne-capoue/63494
13          etienne-capoue/63494
14          etienne-capoue/63494
15          etienne-capoue/63494
16          etienne-capoue/63494
17          etienne-capoue/63494
18          etienne-capoue/63494
19          etienne-capoue/63494
20          etienne-capoue/63494
21          etienne-capoue/63494
22          etienne-capoue/63494
23          etienne-capoue/63494
24          etienne-capoue/63494
25          etienne-capoue/63494
26          etienne-capoue/63494
27          etienne-capoue/63494
28          etienne-capoue/63494
29          etienne-capoue/63494
          

In [82]:
for state in players_stats['names']:
    if "/" in state:
        row_num = players_stats.loc[players_stats['names']==state].index[0]
        players_stats.at[row_num, 'names'] = state[:state.index("/")]
         

In [85]:
players_stats.groupby(players_stats.names).count()

,season,league,apps,PPM,goals,assists,yellow_1/yellow_2/red,mins_played
names,,,,,,,,
abdelazizbarrada,17,17,17,17,17,17,17,17
abdoulaykonko,41,41,41,41,41,41,41,41
abdulkaderkeita,28,28,28,28,28,28,28,28
achilleemana,32,32,32,32,32,32,32,32
adammaher,36,36,36,36,36,36,36,36
adamszalai,27,27,27,27,27,27,27,27
ademljajic,33,33,33,33,33,33,33,33
adilrami,47,47,47,47,47,47,47,47
adrianmierzejewski,45,45,45,45,45,45,45,45


In [ ]:
player.stat

In [145]:
df=pd.DataFrame(data, columns=["season", "league", "apps","PPM","goals","assists", "yellow_1/yellow_2/red", "mins_played"])
df.replace("-","",regex=True, inplace=True)
df.replace("'","",regex=True, inplace=True)
df.drop(["league","PPM","yellow_1/yellow_2/red"], axis=1,inplace=True)
for i in range(len(df.season)):
    df.season.iloc[i]=("20"+df.season.iloc[i][3:])
cols=["apps","goals","assists", "mins_played"]
for i in cols:
    df[i]=pd.to_numeric(df[i])
#df.groupby(['season']).sum().astype(int)